# Pima Indian Diabetes Voorspelling

Importeer enkele basic libraries.
* Panda's - verstrekte dataframes
* matplotlib.pyplot - ondersteuning voor plotten

Gebruik Magic% matplotlib om afbeeldingen inline weer te geven in plaats van in een pop-upvenster.


In [ ]:
import pandas as pd                # pandas is a dataframe library
import matplotlib.pyplot as plt      # matplotlib.pyplot plot data

%matplotlib inline

## Laden en bekijken van Data

In [ ]:
df = pd.read_csv("./data/pima-data.csv")
# df = pd.read_excel('./data/bestandsnaam.xlsx','naam sheet')

In [ ]:
df.shape

In [ ]:
df.head(5)

In [ ]:
df.tail(5)

### Definitie features
Uit de metadata van de databron hebben we de volgende definitie van de features.

| Feature  | Beschrijving | Opmerkingen |
|--------------|-------------|--------|
| num_preg     | aantal zwangerschappen         |
| glucose_conc | Plasmacglucosecconcentratie a 2 uur in een orale glucosectolerantiectest         |
| diastolic_bp | Diastolische bloeddruk (mm Hg) |
| thickness | Triceps huidplooi dikte (mm) |
|insulin | 2-Uur serum insuline (mu U/ml) |
| bmi |  Body mass index (gewicht in kg/(hoogte in m)^2) |
| diab_pred |  Diabetes stamboom functie |
| Age (years) | leeftijd (jaren)|
| skin | ???? | Wat is dit? |
| diabetes | Class variabele (1=True, 0=False) |  Waarom is onze data boolean (True/False)? |


## Controleer null waarde

In [ ]:
df.isnull().values.any()


### Correlated Feature Check

Help functie die correlatie op kleur weergeeft. Geel is het meest gecorreleerd, Blauw het minst.

In [ ]:
def plot_corr(df, size=11):
    """
    De functie zet een grafische correlatiematrix uit voor elk paar kolommen in het dataframe.

     Invoer:
         df: panda's DataFrame
         grootte: verticale en horizontale grootte van het perceel

     Displays:
         matrix van correlatie tussen kolommen. Blauw-cyaan-geel => minder tot meer gecorreleerd
                                                 0 ------------------> 1
                                                 Verwacht een gele lijn die van linksboven naar rechtsonder loopt
    """

    corr = df.corr()    # data frame correlatie functie
    fig, ax = plt.subplots(figsize=(size, size))
    ax.matshow(corr)   # kleur de rechthoeken op correlatiewaarde
    plt.xticks(range(len(corr.columns)), corr.columns)  # Teken maatstrepen x
    plt.yticks(range(len(corr.columns)), corr.columns)  # Teken maatstrepen y


In [ ]:
plot_corr(df)

In [ ]:
df.corr()

In [ ]:
df.head(5)

Huid en dikte zijn 1 op 1 gecorreleerd. Huid kolom laten we vallen.
Gecorreleerde kenmerken verbeteren niet het model. Ze beinvloeden het model op verschillende manieren en mate. Één van de gevolgen kan zijn dat de oplossingen onstabiel worden. 

In [ ]:
del df['skin']

In [ ]:
df.head(5)

Controleer op aanvullende correlaties

In [ ]:
plot_corr(df)

De verbanden zien er goed uit. Er lijken geen gecorreleerde kolommen te zijn.

## Organiseer Data

### Data Types

Inspecteer data typen om te zien of er problemen zijn. De data moet numeriek zijn.

In [ ]:
df.head(5)

Verander diabetes van boolean naar integer, True = 1, False = 0

In [ ]:
diabetes_map = {True : 1, False : 0}
df['diabetes'] = df['diabetes'].map(diabetes_map)

Controleer of het diabetes datatype is gewijzigd.

In [ ]:
df.head(5)

### Check null waarden

In [ ]:
df.isnull().values.any()

Geen duidelijke null-waarden.

### Controleer de klasseverdeling

Zeldzame gebeurtenissen zijn moeilijk te voorspellen

In [ ]:
num_obs = len(df)
num_true = len(df.loc[df['diabetes'] == 1])
num_false = len(df.loc[df['diabetes'] == 0])
print("Aantal True gevallen:  {0} ({1:2.2f}%)".format(num_true, (num_true/num_obs) * 100))
print("Aantal False gevallen: {0} ({1:2.2f}%)".format(num_false, (num_false/num_obs) * 100))

Goede verdeling van True en False. Geen speciaal werk nodig.

### Spliting de data 

70% voor training, 30% voor testing

In [ ]:
from sklearn.model_selection import train_test_split

feature_col_names = ['num_preg', 'glucose_conc', 'diastolic_bp', 'thickness', 'insulin', 'bmi', 'diab_pred', 'age']
predicted_class_names = ['diabetes']

X = df[feature_col_names].values     # voorspellende feature kolommen (8 X m)
y = df[predicted_class_names].values # voorspellende class (1=true, 0=false) kolommen (1 X m)
split_test_size = 0.30

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=split_test_size, random_state=42) 
                            # test_size = 0.3 is 30%, 42 is het antwoord op alles

We controleren of we de gewenste verdeling van 70% train, 30% test van de data hebben

In [ ]:
print("{0:0.2f}% in training set".format((len(X_train)/len(df.index)) * 100))
print("{0:0.2f}% in test set".format((len(X_test)/len(df.index)) * 100))

#### Controleren of de voorspelde waarde correct is gesplitst

In [ ]:
print("Original True  : {0} ({1:0.2f}%)".format(len(df.loc[df['diabetes'] == 1]), (len(df.loc[df['diabetes'] == 1])/len(df.index)) * 100.0))
print("Original False : {0} ({1:0.2f}%)".format(len(df.loc[df['diabetes'] == 0]), (len(df.loc[df['diabetes'] == 0])/len(df.index)) * 100.0))
print("")
print("Training True  : {0} ({1:0.2f}%)".format(len(y_train[y_train[:] == 1]), (len(y_train[y_train[:] == 1])/len(y_train) * 100.0)))
print("Training False : {0} ({1:0.2f}%)".format(len(y_train[y_train[:] == 0]), (len(y_train[y_train[:] == 0])/len(y_train) * 100.0)))
print("")
print("Test True      : {0} ({1:0.2f}%)".format(len(y_test[y_test[:] == 1]), (len(y_test[y_test[:] == 1])/len(y_test) * 100.0)))
print("Test False     : {0} ({1:0.2f}%)".format(len(y_test[y_test[:] == 0]), (len(y_test[y_test[:] == 0])/len(y_test) * 100.0)))


### Voorbereiding van data na de splitsing

#### Verbergen Missing Values

In [ ]:
df.head()

Zijn deze 0 waarden mogelijk?

Hoeveel rijen hebben onverwachte 0 waarden?

In [ ]:
print("# rijen in dataframe {0}".format(len(df)))
print("# rijen missing glucose_conc: {0}".format(len(df.loc[df['glucose_conc'] == 0])))
print("# rijen missing diastolic_bp: {0}".format(len(df.loc[df['diastolic_bp'] == 0])))
print("# rijen missing thickness: {0}".format(len(df.loc[df['thickness'] == 0])))
print("# rijen missing insulin: {0}".format(len(df.loc[df['insulin'] == 0])))
print("# rijen missing bmi: {0}".format(len(df.loc[df['bmi'] == 0])))
print("# rijen missing diab_pred: {0}".format(len(df.loc[df['diab_pred'] == 0])))
print("# rijen missing age: {0}".format(len(df.loc[df['age'] == 0])))

#### Vervang met het gemiddelde

In [ ]:
from sklearn.impute import SimpleImputer

#Vervang alle null waarden met het gemiddelde
fill_0 = SimpleImputer(missing_values=0, strategy="mean") #, axis=0)

X_train = fill_0.fit_transform(X_train)
X_test = fill_0.fit_transform(X_test)

## Training Initial Algorithme - Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

# maak Gaussian Naive Bayes-model object en train het met de data
nb_model = GaussianNB()

nb_model.fit(X_train, y_train.ravel())

### Prestaties op Training Data

In [ ]:
# waarden voorspellen met behulp van de trainings data
nb_predict_train = nb_model.predict(X_train)

# importeer de library met prestatie statistieken
from sklearn import metrics

# Nauwkeurigheid
print("Nauwkeurigheid: {0:.4f}".format(metrics.accuracy_score(y_train, nb_predict_train)))
print()

### Prestaties bij het testen van data

In [ ]:
# waarden voorspellen met behulp van de test data
nb_predict_test = nb_model.predict(X_test)

from sklearn import metrics

# training statistieken
print("Accuracy: {0:.4f}".format(metrics.accuracy_score(y_test, nb_predict_test)))


#### Statistieken

In [ ]:
print("Confusion Matrix")
print("{0}".format(metrics.confusion_matrix(y_test, nb_predict_test)))
print("")

print("Classification Report")
print(metrics.classification_report(y_test, nb_predict_test))


## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(random_state=42, n_estimators=10)      # Creëer random forest object
rf_model.fit(X_train, y_train.ravel()) 

### Voorspel Training Data

In [ ]:
rf_predict_train = rf_model.predict(X_train)
# training statistieken
print("Nauwkeurigheid: {0:.4f}".format(metrics.accuracy_score(y_train, rf_predict_train)))

### Voorspel Test Data

In [ ]:
rf_predict_test = rf_model.predict(X_test)

# training statistieken
print("Nauwkeurigheid: {0:.4f}".format(metrics.accuracy_score(y_test, rf_predict_test)))

In [ ]:
print(metrics.confusion_matrix(y_test, rf_predict_test) )
print("")
print("Classification Report")
print(metrics.classification_report(y_test, rf_predict_test))